In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
pd.options.display.float_format = '${:,.0f}'.format
import requests
import pandas_datareader as dr
import datetime

In [ ]:
#Assumptions

company_ticker = 'WMT'

# Constant Expected Growth Rate
long_term_growth = 0.02

# Market Risk Premium
market_risk_premium = 0.0523

# Risk-free Rate
timespan = 100
current_date = datetime.date.today()
past_date = current_date-datetime.timedelta(days=timespan)
risk_free_rate_df = dr.DataReader('^TNX', 'yahoo', past_date, current_date) 
risk_free_rate = (risk_free_rate_df.iloc[len(risk_free_rate_df)-1,5])/100

#debt spread + fcf growth override
debt_spread = 0.0
fcf_growth_override = 0.4



In [ ]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/cash-flow"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "financial-statements.p.rapidapi.com",
    'x-rapidapi-key': "53aa51e3a0msh4a983fc005dac6cp176d0ajsnb48d0530cd10"
    }

cash_flow_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Cash Flow Statement DataFrame
cash_flow_df = pd.DataFrame.from_dict(cash_flow_response.json())
cash_flow_df = cash_flow_df.drop('ttm', axis = 1)
cash_flow_df = cash_flow_df.astype(float)
cash_flow_df

In [ ]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/income-statement"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "financial-statements.p.rapidapi.com",
    'x-rapidapi-key': "53aa51e3a0msh4a983fc005dac6cp176d0ajsnb48d0530cd10"
    }

income_statement_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Income Statement DataFrame
income_statement_df = pd.DataFrame.from_dict(income_statement_response.json())
income_statement_df = income_statement_df.drop('ttm', axis = 1)
income_statement_df = income_statement_df.astype(float)
income_statement_df

In [ ]:
# FCFF Calculation using Cash Flow Statement and Income Statement Inputs
free_cash_flow_firm = (cash_flow_df.loc['Free Cash Flow'].astype(float) \
                    + (income_statement_df.loc['Interest Expense'].astype(float) \
                       * (1 - income_statement_df.loc['Tax Provision'].astype(float) \
                          / income_statement_df.loc['Pretax Income'].astype(float)))).astype(float)

# Change Series to a Pandas Dataframe
free_cash_flow_firm_df = free_cash_flow_firm.to_frame().transpose()
free_cash_flow_firm_df

In [ ]:
# CAGR of FCFF
latest_free_cash_flow_firm = float(free_cash_flow_firm_df.iloc[0,len(free_cash_flow_firm_df.columns)-1])
earliest_free_cash_flow_firm = float(free_cash_flow_firm_df.iloc[0,1])
free_cash_flow_firm_CAGR = (latest_free_cash_flow_firm/earliest_free_cash_flow_firm)\
                            **(float(1/(len(free_cash_flow_firm_df.columns)-2)))-1

free_cash_flow_firm_CAGR = max(free_cash_flow_firm_CAGR, fcf_growth_override)
print("Free Cash Flow Growth Projection: " + str(free_cash_flow_firm_CAGR*100) +"%")


In [ ]:
# Forecasted FCFF
forecast_free_cash_flow_firm_df = pd.DataFrame(columns=['Year ' + str(i) for i in range(1,7)])
free_cash_flow_firm_forecast_lst = []
for i in range(1,7):
    if i != 6:
        free_cash_flow_firm_forecast = latest_free_cash_flow_firm*(1+free_cash_flow_firm_CAGR)**i
    else:
        free_cash_flow_firm_forecast = latest_free_cash_flow_firm*(1+free_cash_flow_firm_CAGR)\
                                        **(i-1)*(1+long_term_growth)
    free_cash_flow_firm_forecast_lst.append(int(free_cash_flow_firm_forecast))
forecast_free_cash_flow_firm_df.loc[0] = free_cash_flow_firm_forecast_lst
forecast_free_cash_flow_firm_df.astype(float)

In [ ]:
url = "https://stock-analysis.p.rapidapi.com/api/v1/resources/key-stats"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "stock-analysis.p.rapidapi.com",
    'x-rapidapi-key': "53aa51e3a0msh4a983fc005dac6cp176d0ajsnb48d0530cd10"
    }

key_stats_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Key Statistics DataFrame
key_stats_df = pd.DataFrame.from_dict(key_stats_response.json())
key_stats_df = key_stats_df.transpose()
key_stats_df

In [ ]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/balance-sheet"
querystring = {"ticker":company_ticker}
headers = {
    'x-rapidapi-host': "financial-statements.p.rapidapi.com",
    'x-rapidapi-key': "53aa51e3a0msh4a983fc005dac6cp176d0ajsnb48d0530cd10"
    }
balance_sheet_response = requests.request("GET", url, headers=headers, params=querystring)
# Create Balance Sheet DataFrame
balance_sheet_df = pd.DataFrame.from_dict(balance_sheet_response.json())
balance_sheet_df

In [ ]:
# Stock's Beta
equity_beta = float(key_stats_df.loc[r'Beta (5Y monthly)'])
print("Beta: " + str(equity_beta))

In [ ]:
# Required Return on Equity
equity_return = risk_free_rate + (equity_beta*market_risk_premium)
print("Cost of Equity: " +str(equity_return*100) + "%")

In [ ]:
# Interest Expense
interest_expense = income_statement_df.loc['Interest Expense']
interest_expense_df = interest_expense.to_frame().transpose()
interest_expense_str = interest_expense_df.values[0][-1:]
interest_expense_int = int(interest_expense_str)
# Total Debt
total_debt = balance_sheet_df.loc['Total Debt']
total_debt_df = total_debt.to_frame().transpose()
total_debt_str = total_debt_df.values[0][-1:]
total_debt_int = int(total_debt_str)
# Required Return on Debt
debt_return = risk_free_rate + debt_spread

print("Cost of Debt: " + str(float(debt_return*100))+ "%")

In [ ]:
# Effective Tax Rate
effective_tax_rate = income_statement_df.loc['Tax Provision'].astype(int) / income_statement_df.loc['Pretax Income'].astype(int)
avg_effective_tax_rate = sum(effective_tax_rate) / len(effective_tax_rate)
print("Effective Tax Rate: " +str(avg_effective_tax_rate*100) +"%")


In [ ]:
# Market Value of Equity
market_cap_str = key_stats_df.loc[r'Market cap (intra-day)'][0]
market_cap_lst = market_cap_str.split('.')

if market_cap_str[len(market_cap_str)-1] == 'T':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('T',(9-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))
if market_cap_str[len(market_cap_str)-1] == 'B':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('B',(6-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))
if market_cap_str[len(market_cap_str)-1] == 'M':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('M',(3-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))

print("Market Cap = " +'${:,.0f}'.format(market_cap_int))

In [ ]:
# Market Value of Debt
net_debt = balance_sheet_df.loc['Net Debt']
net_debt_df = net_debt.to_frame().transpose()
net_debt_str = net_debt_df.values[0][-1:]
net_debt_int = int(net_debt_str)

print("Net Debt = " +'${:,.0f}'.format(net_debt_int))

In [ ]:
# Company Value
company_value = market_cap_int + net_debt_int
print("Company Value = " +'${:,.0f}'.format(company_value))

In [ ]:
WACC = ((market_cap_int/company_value) * equity_return) \
        + ((net_debt_int/company_value) * (debt_return * (1-avg_effective_tax_rate)))

print("WACC = "+ '{0:,.2f}%'.format(WACC*100))

In [ ]:
# Equity Value Calculation
discounted_FCFF_lst = []
for year in range(0,5):
    discounted_FCFF = forecast_free_cash_flow_firm_df.iloc[0,year]/(1+WACC)**(year+1)
    discounted_FCFF_lst.append(int(discounted_FCFF))
    
terminal_value = forecast_free_cash_flow_firm_df.iloc[0,5]/(WACC-long_term_growth)
print("Terminal Value = " +'${:,.0f}'.format(terminal_value))

PV_terminal_value = int(terminal_value/(1+WACC)**5)
print("PV of terminal Value = " +'${:,.0f}'.format(PV_terminal_value))

enterprise_value = sum(discounted_FCFF_lst)+PV_terminal_value
print("Enterprise Value = " +'${:,.0f}'.format(enterprise_value))


equity_value = enterprise_value-net_debt_int
print("Equity Value = " +'${:,.0f}'.format(equity_value))

In [ ]:
# Total Shares Outstanding
shares_outstanding_str = key_stats_df.loc[r'Shares outstanding '][0]
shares_outstanding_lst = shares_outstanding_str.split('.')


if shares_outstanding_str[len(shares_outstanding_str)-1] == 'T':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('T',(9-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))
if shares_outstanding_str[len(shares_outstanding_str)-1] == 'B':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('B',(6-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))
if shares_outstanding_str[len(shares_outstanding_str)-1] == 'M':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('M',(3-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))

In [ ]:
# Two-stage FCFF Valuation Model Stock Price Estimate
stock_price = equity_value / shares_outstanding_int
stock_price = '${:,.2f}'.format(stock_price)
print("Model Stock Price = %s"%(stock_price))
# Actual Stock Price
actual_stock_price = market_cap_int / shares_outstanding_int
actual_stock_price = '${:,.2f}'.format(actual_stock_price)
print("Actual Stock Price = %s"%(actual_stock_price))

In [ ]:
#Summary Output

print("Company Ticker: " +str(company_ticker))
print("-------------------------------")
print("Output")
print("Actual Stock Price = %s"%(actual_stock_price))
print("Model Stock Price = %s"%(stock_price))
delta = (equity_value / shares_outstanding_int)/(market_cap_int / shares_outstanding_int)- 1
print ("Premimum / (Discount): " +'{:,.2f}%'.format(delta*100))
print("-------------------------------")
print("Financial Summary")


print("Market Cap = " +'${:,.0f}'.format(market_cap_int))
print("Net Debt = " +'${:,.0f}'.format(net_debt_int))
print("Company Value = " +'${:,.0f}'.format(company_value))
print("Shares Outstanding: " +'{:,.0f}'.format(shares_outstanding_int))



print("-------------------------------")
print("Assumptions")
print("Beta: " + str(equity_beta))
print("Free Cash Flow Growth Projection: " + str(free_cash_flow_firm_CAGR*100) +"%")
print("WACC = "+ '{0:,.2f}%'.format(WACC*100))
print("Cost of Equity: " +'{0:,.2f}%'.format(equity_return*100))
print("Cost of Debt: " + '{0:,.2f}%'.format(debt_return*100))
